In [ ]:
import sys
import os
sys.path.insert(0, './../../../DataAndScripts/structured_scripts')

In [2]:
from importlib import reload

import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

import data_analysis as da
import functions as fun
import sims_utils as su
import validate_utils as vu
import load_utils as lu

import ricciardi_class as ric
import network as network

In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/agos/anaconda3/lib/python3.6/site-packages/matplotlib/

/Users/agos/Dropbox/ColumbiaProjects/Code_Reshuffling_Github/1-MainTextFigures/Figure_5_tuan/structured


# Load Best Fit Parameters

In [3]:
data=da.Data_MonkeyMouse('both','./../../../DataAndScripts/experimental_data')

In [4]:
with open("Model_Fit_Separately_StructModel-"+"Best"+'.pkl', 'rb') as handle:
    comb_fit = pickle.load(handle)
    
fit_preds = {}
fit_params = {}
fit_rX = {}
fit_bals = {}
fit_optrs = {}

for anim_idx in range(2):
    fit_preds[anim_idx] = comb_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]]
    fit_params[anim_idx] = np.concatenate(([0,500,0.09,30,1e-3,200],
                                           comb_fit['best_params_'+data.this_animals[anim_idx]]))
    fit_rX[anim_idx] = comb_fit['best_rXs_'+data.this_animals[anim_idx]]
    fit_bals[anim_idx] = comb_fit['full_balance_'+data.this_animals[anim_idx]]
    fit_optrs[anim_idx] = comb_fit['full_opto_ratio_'+data.this_animals[anim_idx]]

# Simulate Best Fit

In [5]:
seeds=[1,3,5,7]
max_min=25

ri = ric.Ricciardi()
ri.set_up_nonlinearity()

In [6]:
struct_rates = [None,None]

for anim_idx in range(2):
    seed_con = fit_params[anim_idx][vu.res_param_idxs_fixed['seed_con']]
    KX       = fit_params[anim_idx][vu.res_param_idxs_fixed['KX']]
    pmax     = fit_params[anim_idx][vu.res_param_idxs_fixed['pmax']]
    SoriE    = fit_params[anim_idx][vu.res_param_idxs_fixed['SoriE']]
    Lam      = fit_params[anim_idx][vu.res_param_idxs_fixed['Lam']]
    Tmax_over_tau_E =fit_params[anim_idx][vu.res_param_idxs_fixed['Tmax_over_tau_E']]

    T = np.arange(0,1.5*Tmax_over_tau_E*ri.tau_E,ri.tau_I/3);
    mask_time=T>(0.5*Tmax_over_tau_E*ri.tau_E)

    J         = fit_params[anim_idx][vu.res_param_idxs['J']]
    GI        = fit_params[anim_idx][vu.res_param_idxs['GI']]
    gE        = fit_params[anim_idx][vu.res_param_idxs['gE']]
    gI        = fit_params[anim_idx][vu.res_param_idxs['gI']]
    beta      = fit_params[anim_idx][vu.res_param_idxs['beta']]
    rX        = fit_rX[anim_idx]

    CV_K      = fit_params[anim_idx][vu.res_param_idxs['CV_K']]
    SlE       = fit_params[anim_idx][vu.res_param_idxs['SlE']]
    SlI       = fit_params[anim_idx][vu.res_param_idxs['SlI']]
    SoriI     = fit_params[anim_idx][vu.res_param_idxs['SoriI']]
    Stun      = fit_params[anim_idx][vu.res_param_idxs['Stun']]
    CV_Lam    = fit_params[anim_idx][vu.res_param_idxs['CV_Lam']]
    L         = fit_params[anim_idx][vu.res_param_idxs['L']]

    stim_size=0.5
    if anim_idx == 0:
        ori_type='saltandpepper'
        print("Animal = Mouse")
    elif anim_idx == 1:
        ori_type='columnar'
        print("Animal = Monkey")
    RF='in'
    tuned='yes'

    net = network.network(seed_con=int(seed_con), n=2, Nl=25, NE=8, gamma=0.25, dl=1,
                          Sl=np.array([[SlE,SlI],[SlE,SlI]]), Sori=np.array([[SoriE,SoriI],[SoriE,SoriI]]),
                          Stun=Stun, ori_type=ori_type)
    net.GI = GI
    
    if anim_idx == 0:
        idxs = np.arange(len(seeds)*net.N)
    elif anim_idx == 1:
        ori_idx=net.get_oriented_neurons(delta_ori=22.5)
        ori_idx=np.hstack([i*net.N+ori_idx for i in range(len(seeds))])

        rf_idx=net.get_centered_neurons(stim_size=0.6)
        rf_idx=np.hstack([i*net.N+rf_idx for i in range(len(seeds))])

        idxs=np.intersect1d(ori_idx,rf_idx)

    print("Parameters used seed= {:d} // GI= {:.2f} // gE= {:.2f} // gI= {:.2f} // beta= {:.2f} // KX= {:d} // pmax= {:.2f}" \
    .format(int(seed_con),GI,gE,gI,beta,int(KX),pmax))
    print("CV_K= {:.4f} // SlE= {:.3f} // SlI= {:.3f} // SoriE= {:.2f} // SoriI= {:.2f} // Stun= {:.2f}"\
    .format(CV_K,SlE,SlI,SoriE,SoriI,Stun))
    print("Lam= {:.3f} // CV_Lam= {:.2f} // J= {:.6f} // rX0= {:.2f} // rX1= {:.2f} // L= {:.2f} // Tmax_over_tau_E= {:d}"\
    .format(Lam,CV_Lam,J,rX[0],rX[1],L,int(Tmax_over_tau_E)))
    print('')
    
    params_dict={}
    params_dict['Stim_Size']=0.5
    params_dict['seed_con']=int(seed_con)
    params_dict['KX']=int(KX)
    params_dict['pmax']=pmax
    params_dict['SoriE']=SoriE
    params_dict['Lam']=Lam
    params_dict['J']=J
    params_dict['GI']=GI
    params_dict['gE']=gE
    params_dict['gI']=gI
    params_dict['beta']=beta
    params_dict['CV_K']=CV_K
    params_dict['SlE']=SlE
    params_dict['SlI']=SlI
    params_dict['SoriI']=SoriI
    params_dict['Stun']=Stun
    params_dict['CV_Lam']=CV_Lam
    params_dict['L']=L

    params_dict['Nl']=25
    params_dict['NE']=8
    params_dict['n']=2
    params_dict['gamma']=0.25
    params_dict['dl']=1

    params_dict['ori_type']=ori_type
    params_dict['vanilla_or_not']=False

    _,_,_,animal_rates,_,_ =\
        lu.sim_const_map(params_dict,rX,ri,T,mask_time,RF,tuned,seed_con,seeds,max_min)
    this_rates = np.zeros((data.nc[anim_idx],2,len(idxs)))
    for idx in range(data.nc[anim_idx]):
        this_rates[idx,:,:] = animal_rates[idx][:,idxs]
        
    struct_rates[anim_idx] = this_rates

Animal = Mouse
Parameters used seed= 0 // GI= 1.80 // gE= 8.09 // gI= 2.58 // beta= 0.34 // KX= 500 // pmax= 0.09
CV_K= 0.0034 // SlE= 0.256 // SlI= 0.136 // SoriE= 30.00 // SoriI= 21.77 // Stun= 48.36
Lam= 0.001 // CV_Lam= 0.71 // J= 0.000093 // rX0= 10.07 // rX1= 10.82 // L= 5.53 // Tmax_over_tau_E= 200

Doing contrast 1 of 7
      Integration stopped after 2.27s of simulation time
      Integration stopped after 0.26s of simulation time
      Integration stopped after 0.48s of simulation time
      Integration stopped after 0.32s of simulation time
      Integration stopped after 0.24s of simulation time
      Integration stopped after 0.23s of simulation time
      Integration stopped after 0.63s of simulation time
Doing contrast 2 of 7
      Integration stopped after 1.3s of simulation time
      Integration stopped after 0.36s of simulation time
      Integration stopped after 0.72s of simulation time
      Integration stopped after 1.4s of simulation time
      Integration stopp

# Save Statistics

In [7]:
for anim_idx in range(2):
    this_f_handle = open('structured_'+data.this_animals[anim_idx]+'_rates.txt','w')
    np.savetxt(this_f_handle,struct_rates[anim_idx].flatten(),fmt='%.6f', delimiter='\t')
    this_f_handle.close()

In [8]:
for anim_idx in range(2):
    this_f_handle = open('structured_'+data.this_animals[anim_idx]+'_preds.txt','w')
    np.savetxt(this_f_handle,fit_preds[anim_idx],fmt='%.6f', delimiter='\t')
    this_f_handle.close()

In [9]:
for anim_idx in range(2):
    this_f_handle = open('structured_'+data.this_animals[anim_idx]+'_bals.txt','w')
    np.savetxt(this_f_handle,fit_bals[anim_idx][:,anim_idx],fmt='%.6f', delimiter='\t')
    this_f_handle.close()

In [10]:
for anim_idx in range(2):
    this_f_handle = open('structured_'+data.this_animals[anim_idx]+'_optrs.txt','w')
    np.savetxt(this_f_handle,fit_optrs[anim_idx][:,anim_idx],fmt='%.6f', delimiter='\t')
    this_f_handle.close()